In [155]:
# Evaluates an n-input, m-output NAND program P on input x
# P is given in the list of tuples representation
def EVAL(n,m,P,x):
    s = len(P)    # no. of lines in the program
    t = 3*len(P)  # maximum no. of unique labels
    avars = [0]*(t*s) # initialize array to 0
    for i in range(n): # initalize inputs to x
        avars[i*t] = x[i]

    for (a,i,b,j,c,k) in P: # evaluate every line of program
        avars[i*t+a] = 1-avars[j*t+b]*avars[k*t+c]

    #  return y_0...y_(m-1) which is
    # avars[1],avars[t+1],...,avars[(m-1)*t+1]
    return [avars[i*t+1] for i in range(m)]


In [3]:
# Converts a NAND program from text to the list of tuples representation
# Assumes a program where no index is larger than the size of the program
def parse(prog):
    avars = { 'x':0, 'y':1 , 'validx':2, 'loop':3 } # dictionary of indices of "workspace" variables
    n = max([int(var[2:]) for var in prog.split() if var[:2]=='x_' ])+1 # no of inputs
    m = max([int(var[2:]) for var in prog.split() if var[:2]=='y_' ])+1 # no of outputs

    def var_idx(vname): # helper local function to return index of named variable
        vname = vname.split('_')
        name = vname[0]
        idx = int(vname[1]) if len(vname)>1 else 0 
        return [avars.setdefault(name,len(avars)),idx]
    
    result = []
    for line in prog.split('\n'):
        if not line or line[0]=='#': continue # ignore empty and commented out lines
        (var1,assign,var2,op,var3) = line.split()
        result.append(var_idx(var1) + var_idx(var2) + var_idx(var3)) 
    return (n,m,result)

In [4]:
# run NAND program prog on input x.
# x can be given either as a list of 0/1 values or as a string of 0's and 1's.
def run(prog,x):
    (n,m,P) = parse(prog)
    if isinstance(x, str):
        return EVAL(n,m,P,[int(c) for c in x])
    return EVAL(n,m,P,x)

In [5]:
def debug(prog,x):
    avars = { 'x':0, 'y':1} # dictionary of indices of "workspace" variables
    names = { 0:'x', 1:'y'}
    n = max([int(var[2:]) for var in prog.split() if var[:2]=='x_' ])+1 # no of inputs
    m = max([int(var[2:]) for var in prog.split() if var[:2]=='y_' ])+1 # no of outputs

    def var_idx(vname): # helper local function to return index of named variable
        vname = vname.split('_')
        name = vname[0]
        idx = int(vname[1]) if len(vname)>1 else 0 
        v = avars.setdefault(name,len(avars))
        names[v] = name
        return [v,idx]
    
    result = []
    linenum = 0
    for line in prog.split('\n'):
        if not line or line[0]=='#': continue # ignore empty and commented out lines
        (var1,assign,var2,op,var3) = line.split()
        result.append([linenum,line, var_idx(var1) + var_idx(var2) + var_idx(var3)]) 
        linenum += 1
    
    if isinstance(x, str):
        x = [int(c) for c in x]

    vals = [[0]*n , [0]*m] + [[0]*(3*len(result))]*(3*len(result))
    vals[0] = x
    for [linenum,line,p] in result:
        (a,i,b,j,c,k) = p
        vals[a][i] = 1-  vals[b][j]*vals[c][k]
        log = ""
        log += "Executing line "+str(linenum)+": "+line+"; "
        log += names[b]+"_"+str(j)+"="+str(vals[b][j])+","
        log += names[c]+"_"+str(k)+"="+str(vals[c][k])+","
        log += names[a]+"_"+str(i)+" is assigned "
        log += str(vals[a][i])
        log += "=NAND("+str(vals[b][j])+","+str(vals[c][k])+")"
        print(log)

    return vals[1]
        

In [6]:
prog = r'''
u   := x_0 NAND x_1
v   := x_0 NAND u
w   := x_1 NAND u
y_0 := v NAND w
y_1 := u NAND u
'''

debug(prog,"11")

Executing line 0: u   := x_0 NAND x_1; x_0=1,x_1=1,u_0 is assigned 0=NAND(1,1)
Executing line 1: v   := x_0 NAND u; x_0=1,u_0=1,v_0 is assigned 1=NAND(1,1)
Executing line 2: w   := x_1 NAND u; x_1=1,u_0=0,w_0 is assigned 0=NAND(1,0)
Executing line 3: y_0 := v NAND w; v_0=0,w_0=0,y_0 is assigned 1=NAND(0,0)
Executing line 4: y_1 := u NAND u; u_0=0,u_0=0,y_1 is assigned 1=NAND(0,0)


[1, 1]

In [34]:
prog= r'''
u   := x_0 NAND x_1
v   := x_0 NAND u
w   := x_1 NAND u
y_0 := v   NAND w
'''

In [8]:
debug(prog,"00")

Executing line 0: u   := x_0 NAND x_1; x_0=0,x_1=0,u_0 is assigned 1=NAND(0,0)
Executing line 1: v   := x_0 NAND u; x_0=0,u_0=1,v_0 is assigned 1=NAND(0,1)
Executing line 2: w   := x_1 NAND u; x_1=0,u_0=1,w_0 is assigned 1=NAND(0,1)
Executing line 3: y_0 := v   NAND w; v_0=1,w_0=1,y_0 is assigned 0=NAND(1,1)


[0]

In [9]:
parse(prog)

(2,
 1,
 [[2, 0, 0, 0, 0, 1],
  [3, 0, 0, 0, 2, 0],
  [4, 0, 0, 1, 2, 0],
  [1, 0, 3, 0, 4, 0]])

In [156]:
EVAL(2,1,[[2, 0, 0, 0, 0, 1],
  [3, 0, 0, 0, 2, 0],
  [4, 0, 0, 1, 2, 0],
  [1, 0, 3, 0, 4, 0]],[1,1])

[0]

In [11]:
run(prog,"11")

[1]

In [12]:
# Converts a NAND++ 
# Converts a reference to variable var to var_0
def parsepp(prog):
    vars = {} # dictionary of indices of "workspace" variables
    
    def var_idx(name): # helper local function to return index of named variable
        if name[:4]=='loop': return [0,0]
        if name[:2]=='x_': return [1,-1 if name[2]=='i' else int(name[2:])]
        if name[:4]=='EOI_': return [2,-1 if name[4]=='i' else int(name[4:])]
        if name[:2]=='y_': return [3,-1 if name[2]=='i' else int(name[2:])]
        vname =  name.split('_')
        if len(vname)==1:
            idx = 0
        elif vname[1]=='i':
            idx = -1
        else:
            idx = int(vname[1])
        return [vars.setdefault(vname[0],4+len(vars)),idx]
    
    result = []
    for line in prog.split('\n'):
        if not line or line[0]=='#': 
            pass # ignore empty and commented out lines
        elif line=='i++':
            result.append([0])
        elif line=='--':
            retult.append([1])
        else:
            (var1,assign,var2,op,var3) = line.split()
            result.append(var_idx(var1)+var_idx(var2)+var_idx(var3)) 
            
    return result

In [13]:
prog = r'''
u    := x_i NAND s
v    := s   NAND u
w    := x_i NAND u
s    := v   NAND w
i++
loop := EOI_i NAND EOI_i
ns   := s   NAND s
y_0  := ns  NAND ns
'''

In [14]:
parsepp(prog)

[[4, 0, 1, -1, 5, 0],
 [6, 0, 5, 0, 4, 0],
 [7, 0, 1, -1, 4, 0],
 [5, 0, 6, 0, 7, 0],
 [0],
 [0, 0, 2, -1, 2, -1],
 [8, 0, 5, 0, 5, 0],
 [3, 0, 8, 0, 8, 0]]

In [15]:

# Evaluates a  NAND++ program P on input x
# P is given in the list of tuples representation
def EVALpp(P,x):
    vars = { 0:[0], 1:x } 
    i = 0
    
    def getval(var,idx):
        if idx==-1: idx = i
        if var==2:  return 1 if idx>= len(x) else 0
        if var in vars and len(vars[var])>idx:
            return vars[var][idx]
        return 0
    
    def setval(var,idx,v):
        if not var in vars:
            vars[var] = []
        if len(vars[var])<= idx:
            vars[var].append([0]*(1+idx-len(vars[var])))
        vars[var][idx] = v
        
    while True:
        for t in P:
            if len(t)==1:
                i = max(0,i+(1-2*t[0]))
            else: 
                setval(t[0],t[1], 1-getval(t[2],t[3])*getval(t[4],t[5]))
        if not getval(0,0): break 
            
    return vars[3]

In [16]:
# Evaluates a  NAND++ program P on input x
# P is given in the list of tuples representation
# (Code is currently buggy)
def EVALpp(P,x):
    C = max([max(t) for t in P])+1 # max variable index used
    vars = [0]*C
    nout = [0]
    i = 0
    
    def vidx(var,idx):
        if idx==-1: idx = i
        v_idx = idx*C+var
        if len(vars) <= v_idx:
            vars.extend([0]*(1+v_idx-len(vars)))
        if var==2 and idx>= len(x):
            vars[v_idx] = 1 
        if var==3 and idx>nout[0]:
            nout[0] = idx
        return v_idx
        
    while True:
        for t in P:
            if len(t)==1:
                i = max(0,i+(1-2*t[0]))
            else: 
                vars[vidx(t[0],t[1])] =  1-vars[vidx(t[2],t[3])]*vars[vidx(t[4],t[5])]
        if not vars[0]: break 

    print(vars)        
    return [vars[C*i+3] for i in range(nout[0]+1)]

In [17]:
EVALpp([[4, 0, 1, -1, 5, 0], 
        [6, 0, 5, 0, 4, 0],
        [7, 0, 1, -1, 4, 0],
        [5, 0, 6, 0, 7, 0],
        [0],
        [0, 0, 2, -1, 2, -1],
        [8, 0, 5, 0, 5, 0],
        [3, 0, 8, 0, 8, 0]],
       [0,0,1,1,0,0,0,1,0])

[0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


[0]

In [18]:
EVALpp([[4, 0, 1, -1, 5, 0], 
        [6, 0, 5, 0, 4, 0],
        [7, 0, 1, -1, 4, 0],
        [5, 0, 6, 0, 7, 0],
        [0],
        
        [8, 0, 5, 0, 5, 0],
        [3, 0, 8, 0, 8, 0]],
       [0,0,1])

[0, 0, 0, 0, 1, 0, 1, 1, 1]


[0]

In [123]:
import re

def tokenize(prog):
    
    scanner=re.Scanner([
      (r"\#.*", None),
      (r"//.*", None),
      (r"\{", lambda scanner,token:("OPEN BRACE",token)),
      (r"\}", lambda scanner,token:("CLOSE BRACE",token)),
      (r"\(", lambda scanner,token:("OPEN PAREN",token)),
      (r"\)", lambda scanner,token:("CLOSE PAREN",token)),
      (r"i\+\+", lambda scanner,token:("INC",token)),
      (r"i\-\-", lambda scanner,token:("DEC",token)),
      (r"i\>\>", lambda scanner,token:("HALVE",token)),
      (r"i\<\<", lambda scanner,token:("DOUBLE",token)),
      (r"i", lambda scanner,token:("INDEX",token)),
      (r"\n", lambda scanner,token:("NEWLINE",token)),  
      (r"[0-9]+",       lambda scanner,token:("INDEX", token)),
      (r"\_",      lambda scanner,token:("UNDERSCORE", token)),  
      (r":=",        lambda scanner,token:("ASSIGN", token)),
      (r"NAND",        lambda scanner,token:("NAND", token)),  
      (r"[a-zA-Z']+",      lambda scanner,token:("NAME", token)),
      (r"\s+", None), # None == skip token.
    ])
    
    res = []
    for line in prog.split('\n'):
        if not line: continue
        temp, remainder=scanner.scan(line)
        results = []
        for i in range(len(temp)):
            results.append(temp[i])
            if   temp[i][0]=='NAME' and (i == len(temp)-1 or temp[i+1][0]!='UNDERSCORE'):
                results += [('UNDERSCORE','_'),('INDEX','*')]
        res.append([line,results])
    return res
    
    
#tokenize(prog)

In [124]:
debug = False
debug_str = ""

In [125]:
def getval(vdict,name,idx):
    global debug    
    global debug_str
    didx = ""
    if idx=='i':
        nidx=vdict['i']
        didx="_<i="+str(nidx)+">"
    elif idx=='*':
        nidx=0
        didx="  "
    else:
        nidx=int(idx)
        didx="_"+str(nidx)
    if not name in vdict:
        vdict[name] = []
    if len(vdict[name])<= nidx:
        vdict[name] += [0]*(nidx+1-len(vdict[name]))
    val = vdict[name][nidx]
    if debug:
        debug_str += name+didx+" = "+str(val)+", "
    return val

def setval(vdict,name,idx,val):
    global debug
    global debug_str
    didx = ""
    if idx=='i':
        nidx=vdict['i']
        didx="_<i="+str(nidx)+">"
    elif idx=='*':
        nidx=0
        didx="  "
    else:
        nidx=int(idx)
        didx="_"+str(nidx)
    if not name in vdict:
        vdict[name] = []
    if len(vdict[name])<= nidx:
        vdict[name] += [0]*(nidx+1-len(vdict[name]))
    vdict[name][nidx] = val
    if debug:
        debug_str += name+didx+" is assigned "+str(val)+", "

def modindex(op,vdict):
    global debug
    global debug_str
    i = vdict['i']
    pc = vdict['prog_counter']
    if debug:
        debug_str += 'i changed from '+str(i)+' to '
    if op=='INC' and i < pc:
        i+=1
    elif op=='DEC' and i>0:
        i-=1
    elif op=='DOUBLE' and i<= pc/2:
        i*= 2
    elif op=='HALVE':
        i = int(i/2)
    if debug:
        debug_str += str(i)+', '
    vdict['i']=i
    
operations = {
('NAME','UNDERSCORE','INDEX','ASSIGN','NAME','UNDERSCORE','INDEX','NAND','NAME','UNDERSCORE','INDEX') : 
    lambda vdict,l: setval(vdict,l[0],l[2],1-getval(vdict,l[4],l[6])*getval(vdict,l[8],l[10])),
('INC','OPEN PAREN','NAME','UNDERSCORE','INDEX','CLOSE PAREN') : 
    lambda vdict,l: modindex('INC',vdict) if getval(vdict,l[2],l[4]) else 0,
    
    
}

In [126]:
def match(key,tokens):
    if len(key) != len(tokens):
        return False
    for i in range(len(key)):
        if key[i] != tokens[i][0]: return False
    return True

def execute(vdict,tokens):
    for k in operations.keys():
        if match(k,tokens):
            # print("INVOKING ON "+str([t[1] for t in tokens]))
            operations[k](vdict,[t[1] for t in tokens])
            break
        
        

In [127]:
def EVAL(prog,x):
    global debug
    global debug_str
    vdict = { 'i':0, 'loop':[0],'x':x , 'prog_counter':0}
    lines = tokenize(prog)
    iters = 1
    while True:
        for (line,tokens) in lines:
            if not line or not tokens: continue
            vdict['prog_counter'] = vdict['prog_counter']+1
            if debug:
                debug_str = 'Executing step '+str(vdict['prog_counter'])+': "'+line+'"\t '
            execute(vdict,tokens)
            if debug:
                print(debug_str)
        if not vdict['loop'][0]:
            break
        else:
            iters += 1
            print("Continuing to iteration number "+str(iters))
    if debug:
        output = 'Output is '
        for i in range(len(vdict['y'])):
            output += 'y_'+str(i)+"="+str(vdict['y'][i])+(', ' if i<len(vdict['y'])-1 else '')
        print (output) 
    return vdict['y']
            

In [137]:
debug = True
prog = r'''
u   := x_0 NAND x_1
v   := x_0 NAND u
w   := x_1 NAND u
y_0 := v NAND w
y_1 := u NAND u
'''

EVAL(prog,[0,1])

Executing step 1: "u   := x_0 NAND x_1"	 x_0 = 0, x_1 = 1, u   is assigned 1, 
Executing step 2: "v   := x_0 NAND u"	 x_0 = 0, u   = 1, v   is assigned 1, 
Executing step 3: "w   := x_1 NAND u"	 x_1 = 1, u   = 1, w   is assigned 0, 
Executing step 4: "y_0 := v NAND w"	 v   = 1, w   = 0, y_0 is assigned 1, 
Executing step 5: "y_1 := u NAND u"	 u   = 1, u   = 1, y_1 is assigned 0, 
Output is y_0=1, y_1=0


[1, 0]

In [153]:
prog=r'''
// Add a pair of two-bit numbers
// Input: (x_0,x_1) and (x_2,x_3)
// Output: (y_0,y_1,y_2) representing the sum
// x_0 + 2x_1 + x_2 + 2x_3
//
// Operation:
// 1) y_0,c_1 := ADD_1(x_0,x_2):
// add the least significant digits
// c_1 is the "carry"
u   := x_0 NAND x_2
v   := x_0 NAND u
w   := x_2 NAND u
y_0 := v NAND w
c_1 := u NAND u
// 2) z'_1,z_1 := ADD_1(x_1,x_3):
// add second digits
u   := x_1 NAND x_3
v   := x_1 NAND u
w   := x_3 NAND u
z_1 := v NAND w
z'_1 := u NAND u
// 3) Take care of carry:
// 3a) y_1 = XOR(z_1,c_1)
u   := z_1 NAND c_1
v   := z_1 NAND u
w   := c_1 NAND u
y_1 := v   NAND w
// 3b) y_2 = z'_1 OR (z_1 AND c_1)
//  = NAND(NOT(z'_1), NAND(z_1,c_1))
u   := z'_1 NAND z'_1
v   := z_1 NAND c_1
y_2 := u NAND v
'''

In [154]:
EVAL(prog,[0,1,1,1])

Executing step 1: "u   := x_0 NAND x_2"	 x_0 = 0, x_2 = 1, u   is assigned 1, 
Executing step 2: "v   := x_0 NAND u"	 x_0 = 0, u   = 1, v   is assigned 1, 
Executing step 3: "w   := x_2 NAND u"	 x_2 = 1, u   = 1, w   is assigned 0, 
Executing step 4: "y_0 := v NAND w"	 v   = 1, w   = 0, y_0 is assigned 1, 
Executing step 5: "c_1 := u NAND u"	 u   = 1, u   = 1, c_1 is assigned 0, 
Executing step 6: "u   := x_1 NAND x_3"	 x_1 = 1, x_3 = 1, u   is assigned 0, 
Executing step 7: "v   := x_1 NAND u"	 x_1 = 1, u   = 0, v   is assigned 1, 
Executing step 8: "w   := x_3 NAND u"	 x_3 = 1, u   = 0, w   is assigned 1, 
Executing step 9: "z_1 := v NAND w"	 v   = 1, w   = 1, z_1 is assigned 0, 
Executing step 10: "z'_1 := u NAND u"	 u   = 0, u   = 0, z'_1 is assigned 1, 
Executing step 11: "u   := z_1 NAND c_1"	 z_1 = 0, c_1 = 0, u   is assigned 1, 
Executing step 12: "v   := z_1 NAND u"	 z_1 = 0, u   = 1, v   is assigned 1, 
Executing step 13: "w   := c_1 NAND u"	 c_1 = 0, u   = 1, w   is assigned

[1, 0, 1]

In [150]:
prog=r'''
nx_2 := x_2 NAND x_2
u    := x_0 NAND nx_2
v    := x_1 NAND x_2
y_0  := u   NAND v   
'''

EVAL(prog,[1,1,1])

Executing step 1: "nx_2 := x_2 NAND x_2"	 x_2 = 1, x_2 = 1, nx_2 is assigned 0, 
Executing step 2: "u    := x_0 NAND nx_2"	 x_0 = 1, nx_2 = 0, u   is assigned 1, 
Executing step 3: "v    := x_1 NAND x_2"	 x_1 = 1, x_2 = 1, v   is assigned 0, 
Executing step 4: "y_0  := u   NAND v   "	 u   = 1, v   = 0, y_0 is assigned 1, 
Output is y_0=1


[1]

In [8]:
import math
def solve_eq(b,c):
    # return solution of x^2 + bx = c using Al Kharizmi's instructions
    val1 = b/2.0 # halve the number of the roots
    val2 = val1*val1 # this you multiply by itself
    val3 = val2 + c # Add this to thirty-nine (c)
    val4 = math.sqrt(val3) # take the root of this
    val5 = val4 - val1 # subtract from it half the number of roots
    return val5  # This is the root of the square which you sought for

In [9]:
solve_eq(10,39)

3.0

In [38]:
def index(prog_counter):
    r = math.floor(math.sqrt(prog_counter+0.25)-0.5)
    s = prog_counter - r*(r+1)
    i = s if s <= r+1 else 2*(r+1)-s
    return (prog_counter,r,s,i)

[index(pc) for pc in range(0,20)]

[(0, 0, 0, 0),
 (1, 0, 1, 1),
 (2, 1, 0, 0),
 (3, 1, 1, 1),
 (4, 1, 2, 2),
 (5, 1, 3, 1),
 (6, 2, 0, 0),
 (7, 2, 1, 1),
 (8, 2, 2, 2),
 (9, 2, 3, 3),
 (10, 2, 4, 2),
 (11, 2, 5, 1),
 (12, 3, 0, 0),
 (13, 3, 1, 1),
 (14, 3, 2, 2),
 (15, 3, 3, 3),
 (16, 3, 4, 4),
 (17, 3, 5, 3),
 (18, 3, 6, 2),
 (19, 3, 7, 1)]

In [23]:
def XOR(a,b):
    return (a+b) % 2

def parity(x):
    seen = [0]*len(x)
    prog_counter = 1
    i = 0
    s = 0
    while i<len(x):
        val = x[i] & (seen[i]==0) 
        s = (s+val) % 2
        seen[i]=1
        r = math.floor(math.sqrt(prog_counter-0.25)-0.5)
        if prog_counter - r*(r+1) > r+1:
            i = i-1
        else:
            i = i+1
        prog_counter = prog_counter+1
    return s

parity([1,0,1,1])

1

In [18]:
indices

[0, 1, 0, 1, 2, 1, 0, 1, 2, 3, 2, 1, 0, 1, 2, 3]

In [19]:
running_sum

[1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0]

In [1]:
2+4+6+8+10+12

42

In [16]:
# Transforms a NAND program to a C function
# prog: string of program code
# n: number of inputs
# m: number of outputs
def nand2C(prog,n,m):
    avars = { } # dictionary of indices of "workspace" variables
    for i in range(n):
        avars['x_'+str(i)] = i
    for j in range(m):
        avars['y_'+str(j)] = n+j
    
    def var_idx(vname): # helper local function to return index of named variable
        vname = vname.split('_')
        name = vname[0]
        idx = int(vname[1]) if len(vname)>1 else 0 
        return avars.setdefault(name+'_'+str(idx),len(avars))
    
    main = "\n"
    
    for line in prog.split('\n'):
        if not line or line[0]=='#' or line[0]=='//': continue # ignore empty and commented out lines
        (var1,assign,var2,op,var3) = line.split()
        main += '    setbit(vars,{idx1}, ~(getbit(vars,{idx2}) & (getbit(vars,{idx2}));\n'.format(
            idx1= var_idx(var1), idx2 = var_idx(var2), idx3 = var_idx(var3))
    
    Cprog = '''
    #include <stdbool.h>
    
    typedef unsigned long bfield_t[ (sizeof(long)-1+{numvars})/sizeof(long) ];
    // From Stackoverflow answer https://stackoverflow.com/questions/2525310/how-to-define-and-work-with-an-array-of-bits-in-c
    // long because that's probably what your cpu is best at
    // The size_needed should be evenly divisable by sizeof(long) or
    // you could (sizeof(long)-1+size_needed)/sizeof(long) to force it to round up
    
    unsigned long getval(bfield_t vars, int idx) {{
        return 1 & (vars[idx / (8 * sizeof(long) )] >> (idx % (8 * sizeof(long)))); 
    }}
    
    void setval(bfield_t vars, int idx, unsigned long v) {{
        vars[idx / (8 * sizeof(long) )] = ((vars[idx / (8 * sizeof(long) )] & ~(1<<b)) | (v<<b); 
    }}
    
    unsigned long int *eval(unsigned long int *x) {{
       bfield_t vars = {{0}};
       int i;
       int j;
       unsigned long int y[{m}] = {{0}};
       for(i=0;i<{n};++i) {{
           setval(vars,i,x[i])
       }}
    '''.format(n=n,m=m,numvars=len(avars))
    
    Cprog = Cprog + main +  '''
        for(j=0;j<{m};++j) {{
           y[j] = getval(vars,{n}+j)
       }}
       return y;
    }}
    '''.format(n=n,m=m)

    return Cprog


In [17]:
prog=r'''
nx_2 := x_2 NAND x_2
u    := x_0 NAND nx_2
v    := x_1 NAND x_2
y_0  := u   NAND v   
'''

print(nand2C(prog,3,1))


    #include <stdbool.h>
    
    typedef unsigned long bfield_t[ (sizeof(long)-1+7)/sizeof(long) ];
    // From Stackoverflow answer https://stackoverflow.com/questions/2525310/how-to-define-and-work-with-an-array-of-bits-in-c
    // long because that's probably what your cpu is best at
    // The size_needed should be evenly divisable by sizeof(long) or
    // you could (sizeof(long)-1+size_needed)/sizeof(long) to force it to round up
    
    unsigned long getval(bfield_t vars, int idx) {
        return 1 & (vars[idx / (8 * sizeof(long) )] >> (idx % (8 * sizeof(long)))); 
    }
    
    void setval(bfield_t vars, int idx, unsigned long v) {
        vars[idx / (8 * sizeof(long) )] = ((vars[idx / (8 * sizeof(long) )] & ~(1<<b)) | (v<<b); 
    }
    
    unsigned long int *eval(unsigned long int *x) {
       bfield_t vars = {0};
       int i;
       int j;
       unsigned long int y[1] = {0};
       for(i=0;i<3;++i) {
           setval(vars,i,x[i])
       }
    
        setbit(vars,4,